In [14]:
from pathlib import Path
import re
from collections import Counter

folder = Path("/Users/ivan/Desktop/Fontys/AML1/AM1-1/data/texts") # write import path/reader yourself as on macOS pathes 
#looks different than on Windows, so generalised path doesnt work properly lol 

books = []

for file in folder.glob("*.txt"):
    with open(file, "r", encoding="utf-8") as f:
        text = f.read()      # read whole file as one string
        books.append(text)   # add entire book as ONE string

print(len(books))           # number of books
print(type(books[0]))       # should be <class 'str'>

94
<class 'str'>


In [15]:
#cleaning the text
print(books[0][:1000])
print(books[0][-1000:])

ï»¿The Project Gutenberg eBook of The Complete Works of William Shakespeare
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Complete Works of William Shakespeare

Author: William Shakespeare

Release date: January 1, 1994 [eBook #100]
                Most recently updated: August 24, 2025

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***




The Complete Works of William Shakespeare

by William Shakespeare




                    Contents

    THE SONNETS
    ALLâS WELL THAT ENDS WELL
    THE TRAGEDY OF ANTONY AND CLEOPAT

In [16]:
START_RE = re.compile(r"\*\*\*\s*START OF (?:THE )?PROJECT GUTENBERG EBOOK.*?\*\*\*", re.IGNORECASE | re.DOTALL)
END_RE   = re.compile(r"\*\*\*\s*END OF (?:THE )?PROJECT GUTENBERG EBOOK.*?\*\*\*", re.IGNORECASE | re.DOTALL)

def clean_gutenberg(text: str) -> str:
    # 1) remove header before START marker - 1st pattern
    m = START_RE.search(text)
    if m:
        text = text[m.end():]

    # 2) remove footer after END marker (license etc.) - 2nd pattern
    m = END_RE.search(text)
    if m:
        text = text[:m.start()]

    # 3) remove common structural patterns that bias stats
    # 3a) chapter headings (roman numerals + digits)
    text = re.sub(r"(?im)^\s*chapter\s+([ivxlcdm]+|\d+)\b.*$", "", text)

    # 3b) collapse excessive blank lines
    text = re.sub(r"\n{3,}", "\n\n", text)

    return text.strip()


cleaned_books = [clean_gutenberg(b) for b in books]

In [17]:
print(cleaned_books[0][:1000])

The Complete Works of William Shakespeare

by William Shakespeare

                    Contents

    THE SONNETS
    ALLâS WELL THAT ENDS WELL
    THE TRAGEDY OF ANTONY AND CLEOPATRA
    AS YOU LIKE IT
    THE COMEDY OF ERRORS
    THE TRAGEDY OF CORIOLANUS
    CYMBELINE
    THE TRAGEDY OF HAMLET, PRINCE OF DENMARK
    THE FIRST PART OF KING HENRY THE FOURTH
    THE SECOND PART OF KING HENRY THE FOURTH
    THE LIFE OF KING HENRY THE FIFTH
    THE FIRST PART OF HENRY THE SIXTH
    THE SECOND PART OF KING HENRY THE SIXTH
    THE THIRD PART OF KING HENRY THE SIXTH
    KING HENRY THE EIGHTH
    THE LIFE AND DEATH OF KING JOHN
    THE TRAGEDY OF JULIUS CAESAR
    THE TRAGEDY OF KING LEAR
    LOVEâS LABOURâS LOST
    THE TRAGEDY OF MACBETH
    MEASURE FOR MEASURE
    THE MERCHANT OF VENICE
    THE MERRY WIVES OF WINDSOR
    A MIDSUMMER NIGHTâS DREAM
    MUCH ADO ABOUT NOTHING
    THE TRAGEDY OF OTHELLO, THE MOOR OF VENICE
    PERICLES, PRINCE OF TYRE
    KING RICHARD THE SECOND
    KI

In [18]:
raw_tokens = []
for text in cleaned_books:
    raw_tokens.extend(text.split())

In [19]:
counter = Counter(raw_tokens)

print("Total tokens:", len(raw_tokens))
print("Unique tokens:", len(counter))
print("Top 20 tokens:", counter.most_common(20))

Total tokens: 13758303
Unique tokens: 472496
Top 20 tokens: [('the', 660536), ('and', 422476), ('of', 390691), ('to', 353059), ('a', 256963), ('in', 222829), ('I', 193344), ('was', 152970), ('that', 150253), ('he', 141079), ('his', 140785), ('with', 112474), ('for', 94901), ('as', 93690), ('had', 92086), ('is', 91305), ('it', 88995), ('not', 85752), ('at', 81835), ('you', 81206)]


In [20]:
#ex2 - BPE tokenizer
from tokenizers import Tokenizer # main object for tokenization
from tokenizers.models import BPE # model implementing Byte Pair Encoding.
from tokenizers.trainers import BpeTrainer # trains the BPE vocabulary
from tokenizers.pre_tokenizers import Whitespace # simple pre-tokenizer splitting on spaces

In [21]:
tokenizer = Tokenizer(BPE())  # BPE model with unknown token
tokenizer.pre_tokenizer = Whitespace() 
trainer = BpeTrainer(vocab_size=20000, min_frequency=1, special_tokens=[], end_of_word_suffix="</w>")
tokenizer.train_from_iterator(cleaned_books, trainer=trainer)

KeyboardInterrupt: 

In [9]:
tokenizer.save("bpe_tokenizer.json")

In [10]:
# Merge all cleaned books into one corpus
all_text = " ".join(cleaned_books)

# Encode entire corpus
encoding = tokenizer.encode(all_text)

# Save token IDs
with open("bpe_token_ids.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(map(str, encoding.ids)))

In [11]:
#ex3 
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter


lemmatizer = WordNetLemmatizer()

# 1) Tokenize all cleaned books into ONE token list (same approach as notebook)
raw_tokens = []
for text in cleaned_books:
    raw_tokens.extend(text.split())

# Optional but recommended: filter out pure punctuation/numbers
# Keeps tokens like "dog" and removes things like "—" or "123"
raw_tokens = [t for t in raw_tokens if any(ch.isalpha() for ch in t)]

# 2) POS tagging (this can take time on many books)
pos_tags = nltk.pos_tag(raw_tokens)

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default

# 3) Lemmatize with lowercasing FIRST (as assignment says)
lemmatized_tokens = [
    lemmatizer.lemmatize(word.lower(), pos=get_wordnet_pos(tag))
    for word, tag in pos_tags
]

# 4) Count + report
lemma_counter = Counter(lemmatized_tokens)

print(len(lemma_counter))
print(lemma_counter.most_common(20))

Unique tokens after lemmatization: 403563
Top 20 lemmatized tokens:
[('the', 712257), ('be', 453824), ('and', 452491), ('of', 397187), ('to', 362566), ('a', 358006), ('in', 234945), ('i', 193367), ('have', 182058), ('he', 177997), ('that', 158162), ('his', 148902), ('it', 117746), ('with', 116791), ('for', 102795), ('you', 89562), ('not', 88061), ('at', 87536), ('my', 84312), ('her', 77033)]


In [12]:
import nltk
from nltk.stem import PorterStemmer
from collections import Counter

[nltk_data] Downloading package punkt to /Users/ivan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
stemmer = PorterStemmer()

# 1) Tokenize all cleaned books into ONE token list
raw_tokens = []
for text in cleaned_books:
    raw_tokens.extend(text.split())

# Optional but recommended: keep only tokens that contain letters
raw_tokens = [t for t in raw_tokens if any(ch.isalpha() for ch in t)]

# 2) Lowercase, then stem
stemmed_tokens = [stemmer.stem(token.lower()) for token in raw_tokens]

# 3) Count + report
stem_counter = Counter(stemmed_tokens)

print(len(stem_counter))
print(stem_counter.most_common(20))

384297
[('the', 712257), ('and', 452495), ('of', 397193), ('to', 362566), ('a', 268463), ('in', 234945), ('i', 193366), ('he', 178013), ('that', 158198), ('wa', 154077), ('hi', 148905), ('it', 117746), ('with', 116791), ('for', 102795), ('as', 101508), ('is', 93971), ('had', 92996), ('you', 89564), ('not', 88079), ('at', 87536)]


less tokens + "be" is not in the list etc
